In [57]:
# Project: Final Team Predictive Models
# COMP_SCI 349-0 Machine Learning | Professor V.S.
# By Sengdao Inthavong, Lillian Torres, Haylie Wu, Ernie Wang

# This file contains the code for the predictive models used in our project.
# The models are trained on the training data and tested on the testing data for the CDs and vinyls.

In [100]:
# Import basic libraries
import os
import numpy as np
import pandas as pd

# Import libraries for text processing
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MinMaxScaler

# Import libraries for machine learning
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB, CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Import libraries for logging
import logging
from tqdm import tqdm

In [101]:
# Load product and review data for CDs and vinyls from the training set
data_dir = ''
categories = ['CDs_and_Vinyl', 'Grocery_and_Gourmet_Food', 'Toys_and_Games']

file_path = os.path.join(data_dir, categories[0], 'train', 'product_training.json')
product_training = pd.read_json(file_path)

file_path = os.path.join(data_dir, categories[0], 'train', 'review_training.json')
review_training = pd.read_json(file_path)

# Merge product and review data
training_data = review_training.merge(product_training, on='asin', how='left')
len(training_data.groupby('asin'))

73082

In [103]:
# Fill in any missing values
training_data['reviewText'].fillna('', inplace=True)
training_data['summary'].fillna('', inplace=True)

In [104]:
# Give each review a unique ID
training_data['reviewID'] = training_data.index


In [105]:
# Run sentiment analysis on the review text and summary
# Columns: neg, neu, pos, compound

# sid = SentimentIntensityAnalyzer()

# review_sentiments = pd.DataFrame(columns=['reviewID', 'reviewText_neg', 'reviewText_neu', 'reviewText_pos', 'reviewText_compound', 'summary_neg', 'summary_neu', 'summary_pos', 'summary_compound'])

# for index, row in tqdm(training_data.iterrows(), total=training_data.shape[0], desc="Sentiment Analysis"):
#     review_text_sentiment = sid.polarity_scores(row['reviewText'])
#     summary_text_sentiment = sid.polarity_scores(row['summary'])
    
#     sentiment_row = {'reviewID': row['reviewID'],
#                      'reviewText_neg': review_text_sentiment['neg'],
#                      'reviewText_neu': review_text_sentiment['neu'],
#                      'reviewText_pos': review_text_sentiment['pos'],
#                      'reviewText_compound': review_text_sentiment['compound'],
#                      'summary_neg': summary_text_sentiment['neg'],
#                      'summary_neu': summary_text_sentiment['neu'],
#                      'summary_pos': summary_text_sentiment['pos'],
#                      'summary_compound': summary_text_sentiment['compound']}
    
#     review_sentiments = review_sentiments.append(sentiment_row, ignore_index=True)

# # Save the sentiment data to a csv file for future use
# file_path = os.path.join(data_dir, categories[0], 'csv', 'review_sentiments.csv')
# review_sentiments.to_csv(file_path, index=False)
review_sentiments = pd.read_csv('review_sentiments.csv')
review_sentiments

,reviewID,reviewText_neg,reviewText_neu,reviewText_pos,reviewText_compound,summary_neg,summary_neu,summary_pos,summary_compound
0,0.0,0.020,0.757,0.223,0.9926,0.0,1.000,0.000,0.0000
1,1.0,0.101,0.822,0.077,0.3222,0.0,1.000,0.000,0.0000
2,2.0,0.088,0.709,0.203,0.7845,0.0,1.000,0.000,0.0000
3,3.0,0.054,0.788,0.158,0.7430,0.0,0.426,0.574,0.4678
4,4.0,0.000,0.196,0.804,0.6249,0.0,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...
770781,770781.0,0.054,0.800,0.146,0.9555,0.0,1.000,0.000,0.0000
770782,770782.0,0.042,0.875,0.083,0.4767,0.0,1.000,0.000,0.0000
770783,770783.0,0.138,0.648,0.214,0.5371,0.0,1.000,0.000,0.0000
770784,770784.0,0.000,0.798,0.202,0.7444,0.0,0.328,0.672,0.6249


In [106]:
# Merge the sentiment data with the training data
training_data = training_data.merge(review_sentiments, on='reviewID', how='left')
training_data.columns

Index(['asin', 'reviewerID', 'unixReviewTime', 'vote', 'verified',
       'reviewTime', 'style', 'reviewerName', 'reviewText', 'summary', 'image',
       'awesomeness', 'reviewID', 'reviewText_neg', 'reviewText_neu',
       'reviewText_pos', 'reviewText_compound', 'summary_neg', 'summary_neu',
       'summary_pos', 'summary_compound'],
      dtype='object')

In [107]:
# Process the columns that are not numeric
encoder = LabelEncoder()

# Define a function to extract the format information from the "style" column
def extract_format(style):
    if style is None:
        return "None"
    else:
        return style.get("Format:", "None").strip()

# Apply the function to the "style" column to extract the format information
training_data["style"] = training_data["style"].apply(extract_format)
training_data

,asin,reviewerID,unixReviewTime,vote,verified,reviewTime,style,reviewerName,reviewText,summary,...,awesomeness,reviewID,reviewText_neg,reviewText_neu,reviewText_pos,reviewText_compound,summary_neg,summary_neu,summary_pos,summary_compound
0,6673F1740E03573BCD64238FE711FC69,9C856D4A18E1355783B3B28B7ECC1848,1451520000,9,True,"12 31, 2015",MP3 Music,8D88BB79AAC50277AEE82FCFD77F6744,Finding the Beatles all over again - and bette...,I sit listening - with my jaw to the floor - H...,...,1,0,0.020,0.757,0.223,0.9926,0.0,1.000,0.000,0.0000
1,690819436E20BB31657AF6B58B984DD4,6E9ABBD26A27C2B2851D1EC34A01CBDC,1113523200,9,False,"04 15, 2005",Audio CD,8DC0611245A871AC51BBEEBB85F33A58,These guys can sing! Such classic tunes...poi...,Under Appreciated....,...,1,1,0.101,0.822,0.077,0.3222,0.0,1.000,0.000,0.0000
2,A680D4753F0CEA2252C168A6ACB2B623,B637C3C93E61094474710F456928BE9F,1126137600,None,False,"09 8, 2005",None,2259386624CFA0EC53A75A50A9BB57A5,"Snoop Doggy Dogg made a classic album, DoggySt...",DoggyStyle,...,1,2,0.088,0.709,0.203,0.7845,0.0,1.000,0.000,0.0000
3,F4A966F1FA340B16651D676BC246D227,AA7918E9410D650A076221C7B2934A09,954979200,None,False,"04 6, 2000",Audio CD,7A65A155C993535BC99CBCB39E7161B5,Stevie Nicks Has had Her Moments. I Like Some ...,Pretty Good but a Bit Dated,...,1,3,0.054,0.788,0.158,0.7430,0.0,0.426,0.574,0.4678
4,EF59DAF0C00319A48D4784266FD157EE,2293C9B7950A3356B95828419A677720,1477958400,None,True,"11 1, 2016",Audio CD,C69A09446009C500B1364B7DB5510497,Great cd.,Five Stars,...,1,4,0.000,0.196,0.804,0.6249,0.0,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770781,BD91503308A437374C3254EDC8BC24CB,936ED23AF4D23943786BBD44D0F1114B,1136246400,12,True,"01 3, 2006",Audio CD,AAB61000438939C8E6165CFCCF02A488,This was the first Simple Minds album that I b...,Their Most Fully Realized Artistic Studio Stat...,...,1,770781,0.054,0.800,0.146,0.9555,0.0,1.000,0.000,0.0000
770782,9BC50277D18FAB423AD33C8CE4CC000D,EF922377A87E9D01F50065F2DA1722A8,1290556800,None,False,"11 24, 2010",None,0441BC4F6B7BD180769FDCDD8E603560,I have owned a CD copy of this show for at lea...,Forgettable R&H,...,0,770782,0.042,0.875,0.083,0.4767,0.0,1.000,0.000,0.0000
770783,4062627CA1586E517520483964299349,E1F0B0EBC6A36F33301E4FD0B3D62D52,1311120000,3,False,"07 20, 2011",Audio CD,8AB3EEBF23F3583A4396A57DB291D548,carnival of souls to me is bad i gave my cd aw...,Darren d.,...,0,770783,0.138,0.648,0.214,0.5371,0.0,1.000,0.000,0.0000
770784,0AE44A6A9176E6A52507B6ABDDA80B00,DDDC81E6B8C3F8C91867F9AECB385135,1111968000,2,False,"03 28, 2005",Audio CD,51B2E62E0A5864316BDB33FD4A729B37,This is an awesome slayer album. I love the th...,awesome slayer cd,...,1,770784,0.000,0.798,0.202,0.7444,0.0,0.328,0.672,0.6249


In [108]:
# Encode the columns
encoder = LabelEncoder()
training_data["style"] = encoder.fit_transform(training_data["style"])

# Encode the "verified" column
training_data["verified"] = encoder.fit_transform(training_data["verified"])

# Encode the "reviewerID" column
training_data["reviewerID"] = encoder.fit_transform(training_data["reviewerID"])

# Encode the "vote"" column
training_data["vote"] = training_data["vote"].apply(lambda x: float(x.replace(",", "")) if x is not None else 0)

# Encode the "image" column
training_data["image"] = training_data["image"].apply(lambda x: len(x) if x is not None else 0)

training_data

,asin,reviewerID,unixReviewTime,vote,verified,reviewTime,style,reviewerName,reviewText,summary,...,awesomeness,reviewID,reviewText_neg,reviewText_neu,reviewText_pos,reviewText_compound,summary_neg,summary_neu,summary_pos,summary_compound
0,6673F1740E03573BCD64238FE711FC69,67931,1451520000,9.0,1,"12 31, 2015",16,8D88BB79AAC50277AEE82FCFD77F6744,Finding the Beatles all over again - and bette...,I sit listening - with my jaw to the floor - H...,...,1,0,0.020,0.757,0.223,0.9926,0.0,1.000,0.000,0.0000
1,690819436E20BB31657AF6B58B984DD4,47952,1113523200,9.0,0,"04 15, 2005",3,8DC0611245A871AC51BBEEBB85F33A58,These guys can sing! Such classic tunes...poi...,Under Appreciated....,...,1,1,0.101,0.822,0.077,0.3222,0.0,1.000,0.000,0.0000
2,A680D4753F0CEA2252C168A6ACB2B623,79174,1126137600,0.0,0,"09 8, 2005",20,2259386624CFA0EC53A75A50A9BB57A5,"Snoop Doggy Dogg made a classic album, DoggySt...",DoggyStyle,...,1,2,0.088,0.709,0.203,0.7845,0.0,1.000,0.000,0.0000
3,F4A966F1FA340B16651D676BC246D227,74058,954979200,0.0,0,"04 6, 2000",3,7A65A155C993535BC99CBCB39E7161B5,Stevie Nicks Has had Her Moments. I Like Some ...,Pretty Good but a Bit Dated,...,1,3,0.054,0.788,0.158,0.7430,0.0,0.426,0.574,0.4678
4,EF59DAF0C00319A48D4784266FD157EE,15169,1477958400,0.0,1,"11 1, 2016",3,C69A09446009C500B1364B7DB5510497,Great cd.,Five Stars,...,1,4,0.000,0.196,0.804,0.6249,0.0,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770781,BD91503308A437374C3254EDC8BC24CB,63910,1136246400,12.0,1,"01 3, 2006",3,AAB61000438939C8E6165CFCCF02A488,This was the first Simple Minds album that I b...,Their Most Fully Realized Artistic Studio Stat...,...,1,770781,0.054,0.800,0.146,0.9555,0.0,1.000,0.000,0.0000
770782,9BC50277D18FAB423AD33C8CE4CC000D,104092,1290556800,0.0,0,"11 24, 2010",20,0441BC4F6B7BD180769FDCDD8E603560,I have owned a CD copy of this show for at lea...,Forgettable R&H,...,0,770782,0.042,0.875,0.083,0.4767,0.0,1.000,0.000,0.0000
770783,4062627CA1586E517520483964299349,98108,1311120000,3.0,0,"07 20, 2011",3,8AB3EEBF23F3583A4396A57DB291D548,carnival of souls to me is bad i gave my cd aw...,Darren d.,...,0,770783,0.138,0.648,0.214,0.5371,0.0,1.000,0.000,0.0000
770784,0AE44A6A9176E6A52507B6ABDDA80B00,96333,1111968000,2.0,0,"03 28, 2005",3,51B2E62E0A5864316BDB33FD4A729B37,This is an awesome slayer album. I love the th...,awesome slayer cd,...,1,770784,0.000,0.798,0.202,0.7444,0.0,0.328,0.672,0.6249


In [109]:
# Compute the length of reviewText and summary columns
training_data["reviewText_len"] = training_data["reviewText"].apply(len)
training_data["summary_len"] = training_data["summary"].apply(len)

In [110]:
training_data
len(training_data.groupby('asin'))

73082

In [33]:
# # Find correlation between reviewText_neg grouped by 0.1 (reviewText_neg group with values 0.0 to 0.1, etc.)
# training_data.groupby(pd.cut(training_data["reviewText_neg"], np.arange(0, 1.1, 0.1))).mean()

In [34]:
# Find correlation between vote grouped by 10
# training_data.groupby(pd.cut(training_data["vote"], np.arange(0, 100, 10))).mean()

In [35]:
# Find correlation between image grouped by 10
# training_data.groupby(pd.cut(training_data["image"], np.arange(0, 50, 5))).mean()

In [36]:
# Find correlation between reviewText_len grouped by 100
# training_data.groupby(pd.cut(training_data["reviewText_len"], np.arange(0, 1000, 100))).mean()

In [37]:
# Find correlation between summary_len grouped by 10
# training_data.groupby(pd.cut(training_data["summary_len"], np.arange(0, 100, 10))).mean()

In [117]:
# from tqdm import tqdm
# import logging

# Group by asin
filteredData = training_data.copy()
filteredData = filteredData[(filteredData["verified"] == 1) | (filteredData["vote"] >= 10) | (filteredData["image"] >= 1)]

# joining filteredData and training_data so we can have at least one review for each product
filteredData = filteredData._append(training_data[~training_data.asin.isin(filteredData.asin)])
# filteredData
len(filteredData)
# len(filteredData.groupby('asin'))
# valid_products = reviews.asin.unique()
# len(valid_products)


770786

In [86]:
# valid_products

array(['6673F1740E03573BCD64238FE711FC69',
       'EF59DAF0C00319A48D4784266FD157EE',
       '7809E907572CA5E262E2F418FD8B3C96', ...,
       '795B5CAF8FD6DAFBC35AD503D32A5CF4',
       '96FD1BD4BC2E152228254DD4A091C7C3',
       '8D02B64BC5D501D4520EF5F459AE189F'], dtype=object)

In [102]:
# results = []
# for asin, reviews in tqdm(training_data.groupby("asin"), total=training_data["asin"].nunique(), desc="Processing Products"):
#     if asin not in valid_products:
#         for _, r in reviews.iterrows():
#             results.append(r)
#     else:
#         newReviews = reviews[(reviews["verified"] == 1) | (reviews["vote"] >= 30) | (reviews["image"] >= 15)]
#         for _, r in newReviews.iterrows():
#             results.append(r)

# results_df = pd.DataFrame(results)
# results_df



# # Normalize the compound scores
# training_data["reviewText_compound_norm"] = (training_data["reviewText_compound"] - training_data["reviewText_compound"].mean()) / training_data["reviewText_compound"].std()
# training_data["summary_compound_norm"] = (training_data["summary_compound"] - training_data["summary_compound"].mean()) / training_data["summary_compound"].std()

# # Calculate the absolute difference between the normalized compound scores and the awesomeness
# training_data["reviewText_compound_diff"] = abs(training_data["reviewText_compound_norm"] - (1 - training_data["reviewText_neg"]))
# training_data["summary_compound_diff"] = abs(training_data["summary_compound_norm"] - (1 - training_data["reviewText_neg"]))

# # Calculate the average difference between the normalized compound scores and the awesomeness for each asin
# compound_diff_mean = training_data.groupby("asin")[["reviewText_compound_diff", "summary_compound_diff"]].mean()

# # Sort the reviews for each asin by the average difference between the normalized compound scores and the awesomeness
# compound_diff_mean["compound_diff_mean"] = compound_diff_mean.mean(axis=1)
# compound_diff_mean = compound_diff_mean.sort_values("compound_diff_mean", ascending=False)

# # Keep the top 2/3 of the reviews for each asin
# num_asins = len(compound_diff_mean)
# top_reviews_per_asin = int(num_asins * 2/3)
# top_asins = compound_diff_mean.iloc[:top_reviews_per_asin].index
# training_data = training_data[training_data["asin"].isin(top_asins)]

TypeError: object of type 'method' has no len()

In [118]:
training_data  = filteredData.copy()
len(training_data)

465913

In [120]:
training_data = training_data.groupby("asin")
len(training_data)

73082

In [94]:
# len(training_data)

70688

In [121]:
# Aggregate the training data by asin
training_data = training_data.agg({
    "reviewerID": "count",
    "unixReviewTime": ["min", "max", "mean", "std"],
    "verified": ["mean", "sum"],
    "vote": ["mean", "sum"],
    "image": ["mean", "sum"],
    "style": ["mean", "sum"],
    "reviewText_neg": ["mean", "std"],
    "reviewText_neu": ["mean", "std"],
    "reviewText_pos": ["mean", "std"],
    "reviewText_compound": ["mean", "std"],
    "summary_neg": ["mean", "std"],
    "summary_neu": ["mean", "std"],
    "summary_pos": ["mean", "std"],
    "summary_compound": ["mean", "std"],
    "reviewText_len": ["mean", "std"],
    "summary_len": ["mean", "std"],
}).reset_index()

training_data

asin reviewerID unixReviewTime               
                                             count            min         max   
0      0000B049F5B33CD310EB1AB236E20191          2     1412294400  1427068800  \
1      00018184A9EC4D270219A296B2580303          4      997488000  1499558400   
2      000281A9CAC43FF1F335726A390636DA          3     1206748800  1417219200   
3      00030884DF109F325638A6BFD5B13CFF         18     1060473600  1496880000   
4      000325BA25966B5FC701D5D2B5DBA4E0          4     1283472000  1456617600   
...                                 ...        ...            ...         ...   
73077  FFFDD3C72D23AF858D6E0ED92612370D         25     1006819200  1507507200   
73078  FFFDDE284A73B29B320381487EC7DE9E          2     1295913600  1506470400   
73079  FFFEB3EE2372807964F024707D50FB21          2     1345852800  1352246400   
73080  FFFF4545AB232D81D0F9B208388BB7AA          4     1417651200  1446076800   
73081  FFFF5A3D9CB0B40FF0FE6B95F05D26FE          3     1097539200  1498780800   

                                   verified          vote        ...   
               mean           std      mean sum      mean   sum  ...   
0      1.419682e+09  1.044708e+07  1.000000   2  0.000000   0.0  ...  \
1      1.333433e+09  2.345825e+08  0.750000   3  5.500000  22.0  ...   
2      1.326499e+09  1.081967e+08  1.000000   3  0.000000   0.0  ...   
3      1.379150e+09  1.140571e+08  0.944444  17  2.388889  43.0  ...   
4      1.363802e+09  7.752298e+07  1.000000   4  1.750000   7.0  ...   
...             ...           ...       ...  ..       ...   ...  ...   
73077  1.394282e+09  1.219028e+08  0.960000  24  1.160000  29.0  ...   
73078  1.401192e+09  1.488861e+08  1.000000   2  1.000000   2.0  ...   
73079  1.349050e+09  4.520958e+06  1.000000   2  0.000000   0.0  ...   
73080  1.434283e+09  1.287727e+07  1.000000   4  0.000000   0.0  ...   
73081  1.231834e+09  2.311845e+08  0.333333   1  9.333333  28.0  ...   

      summary_neu           summary_pos           summary_compound             
             mean       std        mean       std             mean       std   
0        0.293000  0.142836    0.707000  0.142836         0.731850  0.134280  \
1        0.660750  0.417966    0.257250  0.297283         0.199700  0.235639   
2        0.867333  0.229785    0.132667  0.229785         0.170200  0.294795   
3        0.782500  0.323669    0.217500  0.323669         0.236078  0.316375   
4        0.835250  0.329500    0.059500  0.119000        -0.096000  0.192000   
...           ...       ...         ...       ...              ...       ...   
73077    0.700960  0.345688    0.252960  0.347569         0.200232  0.361908   
73078    0.344500  0.045962    0.311500  0.440528         0.107300  0.570352   
73079    0.661500  0.478711    0.177500  0.251023         0.038600  0.054589   
73080    0.633750  0.160672    0.270750  0.066605         0.375075  0.355723   
73081    0.739333  0.451488    0.260667  0.451488         0.185400  0.321122   

      reviewText_len              summary_len             
                mean          std        mean        std  
0         686.000000   115.965512   31.500000  27.577164  
1        1325.750000  2472.037806   21.000000   8.041559  
2         748.000000  1232.756667   20.666667  10.066446  
3         152.388889   238.397133   20.944444  13.531976  
4         469.250000   732.887156   21.750000  14.198005  
...              ...          ...         ...        ...  
73077     710.600000  1204.892768   21.840000  17.973777  
73078     369.500000   482.953932   15.500000   2.121320  
73079     651.500000   129.400541   33.500000   6.363961  
73080     553.750000   226.483811   62.500000  12.974334  
73081     862.666667   660.948056    8.333333   2.886751  

[73082 rows x 34 columns]

In [122]:
# Add +1 to compound columns to avoid negative values
training_data["reviewText_compound"] += 1
training_data["summary_compound"] += 1
# Replace NaN values with 0
training_data.fillna(0, inplace=True)
training_data

asin reviewerID unixReviewTime               
                                             count            min         max   
0      0000B049F5B33CD310EB1AB236E20191          2     1412294400  1427068800  \
1      00018184A9EC4D270219A296B2580303          4      997488000  1499558400   
2      000281A9CAC43FF1F335726A390636DA          3     1206748800  1417219200   
3      00030884DF109F325638A6BFD5B13CFF         18     1060473600  1496880000   
4      000325BA25966B5FC701D5D2B5DBA4E0          4     1283472000  1456617600   
...                                 ...        ...            ...         ...   
73077  FFFDD3C72D23AF858D6E0ED92612370D         25     1006819200  1507507200   
73078  FFFDDE284A73B29B320381487EC7DE9E          2     1295913600  1506470400   
73079  FFFEB3EE2372807964F024707D50FB21          2     1345852800  1352246400   
73080  FFFF4545AB232D81D0F9B208388BB7AA          4     1417651200  1446076800   
73081  FFFF5A3D9CB0B40FF0FE6B95F05D26FE          3     1097539200  1498780800   

                                   verified          vote        ...   
               mean           std      mean sum      mean   sum  ...   
0      1.419682e+09  1.044708e+07  1.000000   2  0.000000   0.0  ...  \
1      1.333433e+09  2.345825e+08  0.750000   3  5.500000  22.0  ...   
2      1.326499e+09  1.081967e+08  1.000000   3  0.000000   0.0  ...   
3      1.379150e+09  1.140571e+08  0.944444  17  2.388889  43.0  ...   
4      1.363802e+09  7.752298e+07  1.000000   4  1.750000   7.0  ...   
...             ...           ...       ...  ..       ...   ...  ...   
73077  1.394282e+09  1.219028e+08  0.960000  24  1.160000  29.0  ...   
73078  1.401192e+09  1.488861e+08  1.000000   2  1.000000   2.0  ...   
73079  1.349050e+09  4.520958e+06  1.000000   2  0.000000   0.0  ...   
73080  1.434283e+09  1.287727e+07  1.000000   4  0.000000   0.0  ...   
73081  1.231834e+09  2.311845e+08  0.333333   1  9.333333  28.0  ...   

      summary_neu           summary_pos           summary_compound             
             mean       std        mean       std             mean       std   
0        0.293000  0.142836    0.707000  0.142836         1.731850  1.134280  \
1        0.660750  0.417966    0.257250  0.297283         1.199700  1.235639   
2        0.867333  0.229785    0.132667  0.229785         1.170200  1.294795   
3        0.782500  0.323669    0.217500  0.323669         1.236078  1.316375   
4        0.835250  0.329500    0.059500  0.119000         0.904000  1.192000   
...           ...       ...         ...       ...              ...       ...   
73077    0.700960  0.345688    0.252960  0.347569         1.200232  1.361908   
73078    0.344500  0.045962    0.311500  0.440528         1.107300  1.570352   
73079    0.661500  0.478711    0.177500  0.251023         1.038600  1.054589   
73080    0.633750  0.160672    0.270750  0.066605         1.375075  1.355723   
73081    0.739333  0.451488    0.260667  0.451488         1.185400  1.321122   

      reviewText_len              summary_len             
                mean          std        mean        std  
0         686.000000   115.965512   31.500000  27.577164  
1        1325.750000  2472.037806   21.000000   8.041559  
2         748.000000  1232.756667   20.666667  10.066446  
3         152.388889   238.397133   20.944444  13.531976  
4         469.250000   732.887156   21.750000  14.198005  
...              ...          ...         ...        ...  
73077     710.600000  1204.892768   21.840000  17.973777  
73078     369.500000   482.953932   15.500000   2.121320  
73079     651.500000   129.400541   33.500000   6.363961  
73080     553.750000   226.483811   62.500000  12.974334  
73081     862.666667   660.948056    8.333333   2.886751  

[73082 rows x 34 columns]

In [123]:
# Define the columns to normalize
cols_to_normalize = [('unixReviewTime', 'max'),
('reviewText_neg', 'mean'),
('reviewText_neg', 'std'),
('reviewText_pos', 'mean'),
('reviewText_pos', 'std'),
('reviewText_compound', 'mean'),
('reviewText_compound', 'std'),
('summary_neg', 'mean'),
('summary_neg', 'std'),
('summary_neu', 'mean'),
('summary_neu', 'std'),
('summary_pos', 'mean'),
('summary_pos', 'std'),
('summary_compound', 'mean'),
('summary_compound', 'std'),
('reviewText_len', 'mean'),
('reviewText_len', 'std'),
('summary_len', 'mean'),
('summary_len', 'std')]

# Normalize the data using the MinMaxScaler
scaler = MinMaxScaler()
training_data[cols_to_normalize] = scaler.fit_transform(training_data[cols_to_normalize])
training_data

asin reviewerID unixReviewTime             
                                             count            min       max   
0      0000B049F5B33CD310EB1AB236E20191          2     1412294400  0.826568  \
1      00018184A9EC4D270219A296B2580303          4      997488000  0.939717   
2      000281A9CAC43FF1F335726A390636DA          3     1206748800  0.811194   
3      00030884DF109F325638A6BFD5B13CFF         18     1060473600  0.935536   
4      000325BA25966B5FC701D5D2B5DBA4E0          4     1283472000  0.872690   
...                                 ...        ...            ...       ...   
73077  FFFDD3C72D23AF858D6E0ED92612370D         25     1006819200  0.952124   
73078  FFFDDE284A73B29B320381487EC7DE9E          2     1295913600  0.950506   
73079  FFFEB3EE2372807964F024707D50FB21          2     1345852800  0.709777   
73080  FFFF4545AB232D81D0F9B208388BB7AA          4     1417651200  0.856237   
73081  FFFF5A3D9CB0B40FF0FE6B95F05D26FE          3     1097539200  0.938503   

                                   verified          vote        ...   
               mean           std      mean sum      mean   sum  ...   
0      1.419682e+09  1.044708e+07  1.000000   2  0.000000   0.0  ...  \
1      1.333433e+09  2.345825e+08  0.750000   3  5.500000  22.0  ...   
2      1.326499e+09  1.081967e+08  1.000000   3  0.000000   0.0  ...   
3      1.379150e+09  1.140571e+08  0.944444  17  2.388889  43.0  ...   
4      1.363802e+09  7.752298e+07  1.000000   4  1.750000   7.0  ...   
...             ...           ...       ...  ..       ...   ...  ...   
73077  1.394282e+09  1.219028e+08  0.960000  24  1.160000  29.0  ...   
73078  1.401192e+09  1.488861e+08  1.000000   2  1.000000   2.0  ...   
73079  1.349050e+09  4.520958e+06  1.000000   2  0.000000   0.0  ...   
73080  1.434283e+09  1.287727e+07  1.000000   4  0.000000   0.0  ...   
73081  1.231834e+09  2.311845e+08  0.333333   1  9.333333  28.0  ...   

      summary_neu           summary_pos           summary_compound             
             mean       std        mean       std             mean       std   
0        0.293000  0.202000    0.707000  0.202000         0.867899  0.515674  \
1        0.660750  0.591093    0.257250  0.420421         0.582426  0.561754   
2        0.867333  0.324966    0.132667  0.324966         0.566601  0.588648   
3        0.782500  0.457737    0.217500  0.457737         0.601941  0.598459   
4        0.835250  0.465983    0.059500  0.168291         0.423797  0.541915   
...           ...       ...         ...       ...              ...       ...   
73077    0.700960  0.488877    0.252960  0.491537         0.582711  0.619160   
73078    0.344500  0.065000    0.311500  0.623000         0.532858  0.713924   
73079    0.661500  0.677000    0.177500  0.355000         0.496003  0.479444   
73080    0.633750  0.227225    0.270750  0.094194         0.676506  0.616348   
73081    0.739333  0.638500    0.260667  0.638500         0.574755  0.600617   

      reviewText_len           summary_len            
                mean       std        mean       std  
0           0.027723  0.005495    0.169355  0.325000  
1           0.053576  0.117134    0.112903  0.094771  
2           0.030228  0.058413    0.111111  0.118634  
3           0.006158  0.011296    0.112605  0.159476  
4           0.018963  0.034727    0.116935  0.167325  
...              ...       ...         ...       ...  
73077       0.028717  0.057092    0.117419  0.211823  
73078       0.014932  0.022884    0.083333  0.025000  
73079       0.026329  0.006131    0.180108  0.075000  
73080       0.022378  0.010732    0.336022  0.152904  
73081       0.034862  0.031318    0.044803  0.034021  

[73082 rows x 34 columns]

In [125]:
# Merge the training data with the awesomeness data
file_path = os.path.join(data_dir, categories[0], 'train', 'product_training.json')
product_training = pd.read_json(file_path)
training_data = training_data.merge(product_training, on='asin', how='left')

# Visualize the absolute correlation between the features on "awesomeness"
training_data.corr()["awesomeness"].abs().sort_values(ascending=False)

MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)

In [ ]:
training_data = training_data.drop(training_data.columns[1], axis=1)

In [ ]:
# Prepare the data for training
# Keep only the most important features for predicting awesomeness
X = training_data[[    
    ('reviewText_neg', 'mean'),
    ('summary_neg', 'std'),
    ('reviewText_pos', 'mean'),
    ('summary_neg', 'mean'),
    ('reviewText_neu', 'mean'),
    ('unixReviewTime', 'max'),
]].values
y = training_data["awesomeness"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train

array([[0.0095    , 0.        , 0.423     , 0.        , 0.5675    ,
        0.85436105],
       [0.01685714, 0.04006423, 0.40164286, 0.00757143, 0.5815    ,
        0.95780933],
       [0.0375    , 0.        , 0.28266667, 0.        , 0.67983333,
        0.90385396],
       ...,
       [0.        , 0.        , 0.3705    , 0.        , 0.6295    ,
        0.86369168],
       [0.00314286, 0.08926525, 0.279     , 0.02385714, 0.71785714,
        0.91629479],
       [0.0125    , 0.        , 0.36025   , 0.        , 0.62725   ,
        0.93887762]])

In [ ]:
# Train the model on naive bayes, decision tree, and random forest classifiers

# Define hyperparameters for each classifier
gnb_params = {
'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}
mnb_params = {
'alpha': [0.1, 0.5, 1.0, 2.0]
}
bnb_params = {
'alpha': [0.1, 0.5, 1.0, 2.0],
'binarize': [0.0, 0.1, 0.5, 1.0]
}
cnb_params = {
'alpha': [0.1, 0.5, 1.0, 2.0]
}

# fit classifiers and make predictions on test set
best_f1_score = 0
best_classifier = None
classifiers = [GaussianNB(), MultinomialNB(), BernoulliNB(), ComplementNB()]
classifier_params = [gnb_params, mnb_params, bnb_params, cnb_params]
classifier_names = ["Gaussian Naive Bayes", "Multinomial Naive Bayes", "Bernoulli Naive Bayes", "Complement Naive Bayes"]

for classifier, params, name in zip(classifiers, classifier_params, classifier_names):
    clf = GridSearchCV(classifier, params, scoring='f1', cv=10)
    with tqdm(total=100, desc=name) as pbar:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

    # compute evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # print results
    print(f"Results for {name}:")
    print(f"Best parameters: {clf.best_params_}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {f1}")
    print(f"Confusion matrix:\n{cm}")

    # check if current classifier is the best one
    if f1 > best_f1_score:
        best_f1_score = f1
        best_classifier = name
        # save the best model
        joblib.dump(clf, best_classifier + '_model.pkl')

print(f"\nBest classifier: {best_classifier} (F1 score: {best_f1_score:.4f})")

Multinomial Naive Bayes:   0%|          | 0/100 [00:00<?, ?it/s]

Results for Gaussian Naive Bayes:
Best parameters: {'var_smoothing': 1e-09}
Accuracy: 0.6065149136577708
Precision: 0.6010826771653544
Recall: 0.8641573510683458
F1 score: 0.7090033087595055
Confusion matrix:
[[1620 4053]
 [ 960 6107]]


Bernoulli Naive Bayes:   0%|          | 0/100 [00:00<?, ?it/s]

Results for Multinomial Naive Bayes:
Best parameters: {'alpha': 1.0}
Accuracy: 0.5942700156985872
Precision: 0.5831143808022421
Recall: 0.9421253714447432
F1 score: 0.7203678658371653
Confusion matrix:
[[ 913 4760]
 [ 409 6658]]


Complement Naive Bayes:   0%|          | 0/100 [00:00<?, ?it/s]

Results for Bernoulli Naive Bayes:
Best parameters: {'alpha': 0.1, 'binarize': 1.0}
Accuracy: 0.5547095761381475
Precision: 0.5547095761381475
Recall: 1.0
F1 score: 0.7135861059221487
Confusion matrix:
[[   0 5673]
 [   0 7067]]


Complement Naive Bayes:   0%|          | 0/100 [00:01<?, ?it/s]

Results for Complement Naive Bayes:
Best parameters: {'alpha': 0.1}
Accuracy: 0.6053375196232339
Precision: 0.6091658635828248
Recall: 0.8050091976793547
F1 score: 0.6935267585029867
Confusion matrix:
[[2023 3650]
 [1378 5689]]

Best classifier: Multinomial Naive Bayes (F1 score: 0.7204)
